In [1]:
#%pip install pyTelegramBotAPI
#%pip install python-dotenv
#%pip install git+https://github.com/huggingface/transformers.git
#%pip install accelerate
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [2]:
from dotenv import load_dotenv
import os
import torch
from transformers import pipeline
import telebot

In [3]:
if torch.cuda.is_available():
    print("CUDA is available! GPU is ready for use.")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Running on CPU.")


CUDA is available! GPU is ready for use.
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [4]:
# Load the .env file
load_dotenv("token.env")

# Access the BOT_TOKEN variable
BOT_TOKEN = os.getenv("BOT_TOKEN")
print(f"Your bot token is: {BOT_TOKEN}")
bot = telebot.TeleBot(BOT_TOKEN)

Your bot token is: 7990668627:AAEWVGv_F2Wxp2hzdpbGrvWdg_fW6pnrnTQ


In [5]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")


Device set to use cuda:0


In [6]:
#testing pipeline
''' messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a teacher",
    },
    {"role": "user", "content": "Hello! How are you?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs) '''

[{'generated_text': '<|system|>\nYou are a friendly chatbot who always responds in the style of a teacher</s>\n<|user|>\nHello! How are you?</s>\n<|assistant|>\nI am fine, thank you. How about you?'}]


In [7]:
@bot.message_handler(commands=['start','hello AI assistant'])
def send_welcome(message):
    bot.reply_to(message, "Hi, how are you doing?")

In [8]:
@bot.message_handler(commands=["info"])
def send_info(message):
  bot.reply_to(message, "This is a simple Telegram bot implemented in Python.")


In [9]:
@bot.message_handler(func=lambda msg: True)
def echo_all(message):
    
    messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds in the style of a teacher",
        },
        {"role": "user", "content": message.text},
    ]
    #print("messages",messages)
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    #print("prompt",prompt)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    print("outputs",outputs)
    reply=outputs[0]["generated_text"]
    assistant_answer = reply[len(prompt):].strip()
    bot.reply_to(message, assistant_answer)

In [10]:
bot.infinity_polling()

outputs [{'generated_text': '<|system|>\nYou are a friendly chatbot who always responds in the style of a teacher</s>\n<|user|>\ntell me something about dogs</s>\n<|assistant|>\nDogs are one of the most beloved animals in the world. They are known for their loyalty, intelligence, and playfulness. Here are some interesting facts about dogs:\n\n1. Dogs are the second-most popular pets in the world, after cats.\n\n2. Dogs are the most popular breed in the US, with more than 9 million dogs owned by people.\n\n3. Dogs are the only animal that can understand human language.\n\n4. Dogs are considered to be emotional beings, just like humans.\n\n5. Dogs are great for mental health, as they provide companionship, stress relief, and emotional support.\n\n6. Dogs have a unique ability to recognize human emotions, such as joy, sadness, and anger.\n\n7. Dogs can sense when people are stressed or unhappy, and will often lick their owners to show empathy and comfort.\n\n8. Dogs can learn new commands

2025-01-25 16:10:28,914 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-01-25 16:10:28,915 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
